In [ ]:
import Pkg 
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
import DifferentiableCollisions as dc 
import MeshCat as mc 
using StaticArrays 

# MeshCat Guide

## Resources

MeshCat docs: https://github.com/rdeits/MeshCat.jl/tree/master/notebooks

URDF support: https://github.com/JuliaRobotics/MeshCatMechanisms.jl

animate cartpole example: https://github.com/Optimal-Control-16-745/HW3_S23_solutions/blob/main/utils/cartpole_animation.jl

animation with mesh example: https://github.com/Optimal-Control-16-745/HW2_S23_solutions/blob/main/utils/rendezvous.jl#L54

DifferentiableCollisions.jl: https://github.com/kevin-tracy/DifferentiableCollisions.jl

Examples using dc: https://github.com/kevin-tracy/DifferentiableCollisions.jl/tree/master/examples

### Static Arrays 

`DifferentiableCollisions.jl` uses StaticArrays. This means that we can't just use our normal vectors, we have to convert them to `SVector`'s first by using `SVector{vector_length}(normal_vector)` like so:

```julia 
using StaticArrays

normal_vector = [1,2,3,4.0]

static_vector = SVector{4}(normal_vector)
```

## Simple Animations in MeshCat with `DifferentiableCollisions.jl`

You can use any of these shapes:
```julia
polytope = dc.Polytope(A, b)   # polytope is described by Ax <= b
capsule  = dc.Capsule(R, L)    # radius R, length L
cylinder = dc.Cylinder(R, L)   # radius R, length L
cone     = dc.Cone(H, β)       # height H, half angle β
sphere   = dc.Sphere(R)        # radius R
ellips   = dc.Ellipsoid(P)     # x'*P*x ≦ 1
polygon  = dc.Polygon(A, b, R) # polygon is described by Ay <= b, cushion radius R
```

Each shape has a position `r` and attitude quaternion `q`. We can update these by doing the following:

```julia
polytope.r = SVector{3}(randn(3)) # update with a random position 
polytope.q = normalize(SVector{4}(randn(4))) # random quaternion, normalized 
```

check out https://github.com/kevin-tracy/DifferentiableCollisions.jl for more info. 

In [ ]:
function create_fake_trajectory(N)
    # create a fake trajectory for N steps
    
    # linearly interpolate between two random positions 
    r1 = 5*randn(3)
    r2 = 5*randn(3)
    rs = range(r1,r2, length = N)
    
    # do the same thing with attitude using MRP's 
    p1 = .5*randn(3)
    p2 = .5*randn(3)
    ps = range(p1,p2, length = N)
    
    # convert to quaternion (convert to StaticVector first)
    qs = [dc.q_from_mrp(SVector{3}(p)) for p in ps]
    
    return rs, qs 
end

let 
    
    # start meshcat visualizer 
    vis = mc.Visualizer()
    
    # create capsule 
    capsule = dc.Capsule(1.4, 3.0)    
    dc.build_primitive!(vis, capsule, :capsule; color = mc.RGBA(1,0,0,1.0))
    
    # length of trajectory 
    dt = 0.1 
    tf = 5.0 
    t_vec = 0:dt:tf 
    N = length(t_vec)
    rs, qs = create_fake_trajectory(N)
    
    # animation 
    anim = mc.Animation(floor(Int,1/dt))
    for k = 1:length(rs)
        mc.atframe(anim, k) do
            
            # update position and attitude of capsule in DifferentiableCollisions
            capsule.r = SVector{3}(rs[k]) # position in world frame W
            capsule.q = SVector{4}(qs[k]) # quaternion ᵂqᴮ
            
            # update position and attitude in MeshCat
            dc.update_pose!(vis[:capsule], capsule)
            
        end
    end
    mc.setanimation!(vis, anim)
    
    display(mc.render(vis))
    
end

## Recording

You can record an animation by going to `Open Control` -> `Recording`, and it will give you instructions on how to convert the resulting images into a video using [ffmpeg](https://ffmpeg.org/). If you have a Mac with an ARM chip, you should use the following command:

```
ffmpeg -r 60 -i %07d.png -vcodec libx264 -pix_fmt yuv420p -preset slow -crf 18 output2.mp4
```

Alternatively, you can just use a screen recording tool.